In [36]:
import logging
import os
from datetime import datetime

# Create logs directory
log_dir = '../outputs/logs'
os.makedirs(log_dir, exist_ok=True)

# Create unique log filename with timestamp (to ms)
timestamp = datetime.now().strftime('%Y-%m-%d_%H-%M-%S-%f')[:-3]
log_filename = f'run_{timestamp}.log'
log_path = os.path.join(log_dir, log_filename)

# 🔁 Remove all handlers associated with the root logger
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# ✅ Reconfigure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s | %(levelname)s | %(message)s',
    handlers=[
        logging.FileHandler(log_path),
        logging.StreamHandler()  # Optional: also print to console
    ]
)

logging.info("🚀 Logging system initialized.")


--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\logging\__init__.py", line 1086, in emit
    stream.write(msg + self.terminator)
UnicodeEncodeError: 'cp950' codec can't encode character '\U0001f680' in position 33: illegal multibyte sequence
Call stack:
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Alex\TUM\hackathon\hackathon-data\HackathonChallengeOne\venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Alex\TUM\hackathon\hackathon-data\HackathonChallengeOne\venv\lib\site-packages\traitlets

In [37]:
# Import pandas for data handling
import pandas as pd

# Import NLTK stopwords for German text preprocessing
import nltk
from nltk.corpus import stopwords

from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [38]:
# Download German stopwords if not already available
try:
    stopwords.words("german")
except LookupError:
    nltk.download("stopwords")

# Store German stopwords for use in TF-IDF vectorizer
german_stopwords = stopwords.words("german")

In [39]:
# Load training and validation datasets from CSV files
train_df = pd.read_csv('../data/challenge_1/train/classification_data.csv')
val_df = pd.read_csv('../data/challenge_1/val/classification_data.csv')

full_df = pd.concat([train_df, val_df], ignore_index=True)

In [40]:
# Features to use
text_feature = "description"

# Define additional binary keyword indicator features
keyword_features = [
    "has_verkehr_keywords",
    "has_bildung_keywords",
    "has_umwelt_keywords",
    "has_gesundheit_keywords"
]

In [41]:
# Select features and labels from the training and validation datasets
X_train = train_df[[text_feature] + keyword_features]
y_train = train_df["responsible_entity_id"]

X_val = val_df[[text_feature] + keyword_features]
y_val = val_df["responsible_entity_id"]


In [42]:
# Limit the classification task to the top N most frequent categories
#TOP_N = 20
#top_classes = y_train.value_counts().nlargest(TOP_N).index

# Filter training data to only include top N classes
#train_mask = y_train.isin(top_classes)
#X_train_filtered = X_train[train_mask]
#y_train_filtered = y_train[train_mask]

# Filter validation data to only include same top N classes
#val_mask = y_val.isin(top_classes)
#X_val_filtered = X_val[val_mask]
#y_val_filtered = y_val[val_mask]

# Use all training and validation data as-is
X_train_filtered = X_train
y_train_filtered = y_train

X_val_filtered = X_val
y_val_filtered = y_val




In [43]:
from sklearn.model_selection import train_test_split


from sklearn.model_selection import train_test_split

X = full_df[[text_feature] + keyword_features]
y = full_df["responsible_entity_id"]


# Count the number of samples per class
class_counts = y.value_counts()

# Select only classes with at least 2 samples
valid_classes = class_counts[class_counts >= 2].index

# Filter X and y to keep only valid classes
X_valid = X[y.isin(valid_classes)]
y_valid = y[y.isin(valid_classes)]

# Now, you can safely stratify on y_valid
X_train, X_val, y_train, y_val = train_test_split(
    X_valid, y_valid,
    test_size=0.2,
    random_state=42,
    stratify=y_valid
)

X_train_filtered_new = X_train
y_train_filtered_new = y_train

X_val_filtered_new = X_val
y_val_filtered_new = y_val



In [44]:
# Create a text processing pipeline: TF-IDF vectorization + Dimensionality Reduction
text_pipeline = Pipeline([
    ("tfidf", TfidfVectorizer(
        stop_words=german_stopwords,
        max_features=10000,
        ngram_range=(1, 2),
        min_df=3,
        max_df=0.9
    )),
    ("svd", TruncatedSVD(n_components=300, random_state=42))  # Reduce to 300 dimensions
])

# Combine text and keyword features into a single preprocessing pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("text", text_pipeline, "description"),
        ("keywords", "passthrough", keyword_features),
    ]
)

In [45]:
# Create full pipeline: preprocessing + classifier
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("clf", RandomForestClassifier(
    n_estimators=100,
    class_weight="balanced",
    random_state=42,
    n_jobs=-1
))
])

In [46]:
# Fit the model on the filtered training data
pipeline.fit(X_train_filtered_new, y_train_filtered_new)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('text',
                                                  Pipeline(steps=[('tfidf',
                                                                   TfidfVectorizer(max_df=0.9,
                                                                                   max_features=10000,
                                                                                   min_df=3,
                                                                                   ngram_range=(1,
                                                                                                2),
                                                                                   stop_words=['aber',
                                                                                               'alle',
                                                                                               'allem',
                                                                                               'allen',
                                                                                               'aller',
                                                                                               'alles',
                                                                                               'als',
                                                                                               'also',
                                                                                               'am',
                                                                                               'an',
                                                                                               'ander',
                                                                                               'andere',
                                                                                               'anderem',
                                                                                               'anderen',
                                                                                               'anderer',
                                                                                               'anderes',
                                                                                               'anderm',
                                                                                               'andern',
                                                                                               'anderr',
                                                                                               'anders',
                                                                                               'auch',
                                                                                               'auf',
                                                                                               'aus',
                                                                                               'bei',
                                                                                               'bin',
                                                                                               'bis',
                                                                                               'bist',
                                                                                               'da',
                                                                                               'damit',
                                                                                               'dann', ...])),
                                                                  ('svd',
                                                                   TruncatedSVD(n_components=300,
                                                         

In [47]:
# Save the trained pipeline model to /outputs/models
import joblib
import os

model_output_dir = '../outputs/models/'
os.makedirs(model_output_dir, exist_ok=True)

model_path = os.path.join(model_output_dir, 'challenge1_model.pkl')
joblib.dump(pipeline, model_path)

logging.info(f"✅ Model saved to: {model_path}")

--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\logging\__init__.py", line 1086, in emit
    stream.write(msg + self.terminator)
UnicodeEncodeError: 'cp950' codec can't encode character '\u2705' in position 33: illegal multibyte sequence
Call stack:
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Alex\TUM\hackathon\hackathon-data\HackathonChallengeOne\venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Alex\TUM\hackathon\hackathon-data\HackathonChallengeOne\venv\lib\site-packages\traitlets\con

In [48]:
# Generate predictions on the filtered validation set
y_pred_filtered = pipeline.predict(X_val_filtered_new)

In [49]:
# Print overall accuracy
logging.info("⭐ Accuracy:", accuracy_score(y_val_filtered_new, y_pred_filtered))

# Print detailed classification report with zero_division=0 to suppress warnings
logging.info(classification_report(y_val_filtered_new, y_pred_filtered, zero_division=0))

--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\logging\__init__.py", line 1083, in emit
    msg = self.format(record)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\logging\__init__.py", line 927, in format
    return fmt.format(record)
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\logging\__init__.py", line 663, in format
    record.message = record.getMessage()
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\logging\__init__.py", line 367, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Call stack:
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 197, in _run_modul

In [50]:
import os
import pandas as pd

# Load test data
test_df = pd.read_csv('../data/challenge_1/test/classification_data.csv')

# Select the same features as used during training
text_feature = "description"
keyword_features = [
    "has_verkehr_keywords",
    "has_bildung_keywords",
    "has_umwelt_keywords",
    "has_gesundheit_keywords"
]
test_X = test_df[[text_feature] + keyword_features]

# Make predictions using the trained pipeline
predictions = pipeline.predict(test_X)

# Create submission DataFrame
submission = pd.DataFrame({
    'issue_id': test_df['issue_id'],
    'responsible_entity_id': predictions
})

# ✅ Ensure the output directory exists
output_dir = '../outputs/submission/'
os.makedirs(output_dir, exist_ok=True)

# ✅ Save submission file in the correct directory
submission_path = os.path.join(output_dir, 'challenge1_submission_new.csv')
submission.to_csv(submission_path, index=False, encoding='utf-8')

# Optional: Save test data with predictions for inspection
debug_path = os.path.join(output_dir, 'test_with_predictions_new.csv')
test_df["predicted_responsible_entity_id"] = predictions
test_df.to_csv(debug_path, index=False, encoding='utf-8')

logging.info(f"✅ Submission saved to: {submission_path}")
logging.info(f"📄 Full test data with predictions saved to: {debug_path}")

--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\logging\__init__.py", line 1086, in emit
    stream.write(msg + self.terminator)
UnicodeEncodeError: 'cp950' codec can't encode character '\u2705' in position 33: illegal multibyte sequence
Call stack:
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Alex\TUM\hackathon\hackathon-data\HackathonChallengeOne\venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Alex\TUM\hackathon\hackathon-data\HackathonChallengeOne\venv\lib\site-packages\traitlets\con

In [51]:
# Logging is done
try:
    # your main code here
    logging.info('🎉 Execution completed successfully.')
except Exception as e:
    logging.error(f"❌ Execution failed: {e}", exc_info=True)


--- Logging error ---
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\logging\__init__.py", line 1086, in emit
    stream.write(msg + self.terminator)
UnicodeEncodeError: 'cp950' codec can't encode character '\U0001f389' in position 33: illegal multibyte sequence
Call stack:
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Alex\TUM\hackathon\hackathon-data\HackathonChallengeOne\venv\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Alex\TUM\hackathon\hackathon-data\HackathonChallengeOne\venv\lib\site-packages\traitlets